In [1]:
import shap
import pickle
import torch
import numpy as np

In [2]:
with open("data/shap_data_pan16.pkl", "rb") as f:
    data_dict = pickle.load(f)

with open("data/shap_values_pan16.pkl", "rb") as f:
    shap_values_task, shap_values_adv, shap_values_adv_mod, shap_values_adv_mod_seq = pickle.load(f)

In [3]:
shap_dict = {
    "task": shap_values_task,
    "adv": shap_values_adv,
    "adv_mod": shap_values_adv_mod,
    "adv_mod_seq": shap_values_adv_mod_seq
}

In [4]:
list(data_dict.keys())

['inputs',
 'emb',
 'emb_deb',
 'emb_deb_mod',
 'emb_deb_mod_seq',
 'task_labels',
 'task_pred',
 'task_deb_pred',
 'task_deb_mod_pred',
 'task_deb_mod_seq_pred',
 'prot_labels',
 'prot_pred',
 'prot_mod_pred',
 'prot_mod_seq_pred',
 'inputs_text',
 'attack_pred',
 'attack_deb_pred',
 'attack_deb_mod_pred',
 'attack_deb_mod_pred_seq']

In [9]:
idx = 8385
keys = [
    'inputs_text',
    'task_labels',
    'task_pred',
    'task_deb_pred',
    'task_deb_mod_pred',
    'task_deb_mod_seq_pred',
    'prot_labels',
    'prot_pred',
    'prot_mod_pred',
    'prot_mod_seq_pred'
]
for k in keys:
    v = data_dict[k]
    print(f"{k}: {v[idx]}")

inputs_text: photoset : hi internet i am looking for a roommate. email me at amandalynferri if you want to hang....
task_labels: 1
task_pred: 4.593868255615234
task_deb_pred: 6.865817546844482
task_deb_mod_pred: 9.694220542907715
task_deb_mod_seq_pred: 6.637912750244141
prot_labels: 0
prot_pred: 1
prot_mod_pred: 1
prot_mod_seq_pred: 1


In [23]:
import torch
import shap
from functools import partial

In [15]:
data_dict["emb"][:100].shape

torch.Size([100, 768])

In [25]:
partial(f, m=l)

TypeError: the first argument must be callable

In [33]:
def fn(x, m):
    return m(torch.from_numpy(x)).detach().numpy()

l = torch.nn.Linear(768, 10)
X = data_dict["emb"].numpy()[:1000]

explainer = shap.KernelExplainer(partial(fn, m=l), X)
shap_values = explainer.shap_values(X)

Using 1000 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [9]:
keys_to_compare = ["attack_pred", "attack_deb_pred", "attack_deb_mod_pred", "attack_deb_mod_pred_seq"]
compare = torch.stack([data_dict[k] for k in keys_to_compare]).sum(0)
f_idx = (~torch.logical_or(compare == 0, compare == len(keys_to_compare))).nonzero(as_tuple=True)[0]

In [16]:
keys_to_compare = ["attack_pred", "attack_deb_pred", "attack_deb_mod_pred", "attack_deb_mod_pred_seq"]
compare = torch.stack([data_dict[k] for k in keys_to_compare])

In [49]:
tmp = compare[1:].sum(0)
a = torch.logical_and(~compare[0].bool(), tmp == 3)
b = torch.logical_and(compare[0].bool(), tmp == 0)
c = torch.logical_or(a,b)
f_idx = c.nonzero(as_tuple=True)[0]

In [52]:
np.random.choice(f_idx, size=10)

array([ 99051, 149560,  82734,  92763, 124119, 101173,  34352,  63831,
         4077,  24913])

In [13]:
# ds = {
#     "labels": data_dict["task_labels"][f_idx],
#     "text": [data_dict["inputs_text"][i] for i in f_idx]
# }

In [54]:
idx = np.random.choice(f_idx, size=5)
for i in idx:
    for k,v in shap_dict.items():
        with open(f'charts/shap_viz_{i}_{k}.html','w') as f:
                f.write(shap.plots.text(v[i], display=False))